## Developing an Information Retrieval System with Spelling Correction and Wildcard Queries

This project aims to enhance the Information Retrieval (IR) system developed in the first assignment by handling
Spelling Correction and Wildcard Queries. This assignment can be completed independently of Project 1. You can
find the data here.


### 1. Document Preprocessing

Your project will begin by reading and preprocessing a collection of text documents. You only need to refer to the dataset as a word list.


In [409]:
import os

import re  # Only for preprocessing

import numpy as np

from collections import deque

In [410]:
docs_filenames = os.listdir(path="docs")

In [411]:
def clean_text(text):
    """Lowercasing and removing extra characters."""

    text = text.lower()  # Lowercasing

    text = re.sub("[^a-z0-9\s\-]", "", text)  # Removing punctuations

    text = re.sub("\-", " ", text)  # Replacing dash with space

    return text

In [412]:
words_set = set()

for filename in docs_filenames:
    with open(f"docs/{filename}") as f:
        text = f.readline()

        text = clean_text(text)

        words_set.update(text.split())

print(len(words_set))

words_set

1348


{'private',
 'scoop',
 'check',
 'our',
 'gallon',
 'ended',
 'reason',
 'cheaper',
 'ill',
 'middle',
 'idea',
 'busjerry',
 '30',
 'having',
 'quell',
 'joy',
 'gangster',
 'libraries',
 'seashell',
 'reduced',
 'autograph',
 'ending',
 'just',
 'persons',
 'made',
 'clones',
 'lane',
 'hauled',
 'wash',
 'find',
 'worked',
 'exact',
 'parking',
 '70',
 'economy',
 'streambed',
 'accidentally',
 'got',
 'homebuyers',
 'allen',
 'delightful',
 'tim',
 'getting',
 'plastic',
 'tuner',
 'legal',
 'months',
 'chasethey',
 'sell',
 'lucky',
 'thing',
 'prison',
 'refused',
 'wearing',
 'downtown',
 'market',
 '10',
 'saved',
 'antennas',
 'often',
 'fines',
 'carriages',
 'put',
 'altadena',
 'numbers',
 'sizes',
 'trade',
 'whose',
 'greater',
 'line',
 'saw',
 'then',
 'until',
 'near',
 'reads',
 'empty',
 'two',
 'complications',
 'other',
 'theyd',
 'variable',
 'half',
 'cost',
 'roof',
 'insurance',
 'burn',
 'ruined',
 'during',
 'attendance',
 'asked',
 'notify',
 'it',
 'first',

### 2. Spelling Correction:

You will implement a function for isolated spelling correction. Your function needs to
correct an input query using Levenshtein distance based on the words in the list. As the word list derived from
the data is not complete, your function does not work flawlessly for all input queries.


In [413]:
def levenshtein_distance(word1, word2):
    m = np.zeros((len(word1) + 1, len(word2) + 1))

    for i in range(len(word1) + 1):
        m[i, 0] = i

    for j in range(len(word2) + 1):
        m[0, j] = j

    for i in range(1, len(word1) + 1):
        for j in range(1, len(word2) + 1):
            m[i, j] = min(
                m[i - 1, j] + 1,
                m[i, j - 1] + 1,
                m[i - 1, j - 1] + (0 if word1[i-1] == word2[j-1] else 1),
            )

    return m[len(word1), len(word2)]

In [414]:
levenshtein_distance("oslo", "snow")

3.0

In [415]:
def find_nearest_word(word, words_set):
    """Find nearest word using Levenshtein distance method"""

    min_distance = float("inf")
    nearest_word = None

    for w in words_set:
        distance = levenshtein_distance(word, w)

        if distance < min_distance:
            min_distance = distance
            nearest_word = w

        if distance == 0:
            break

    return nearest_word

In [416]:
find_nearest_word("festivsl", words_set)

'festival'

In [417]:
def spell_checking(query, words_set):
    """Correct the query using words_set and Levenshtein distance method"""

    query = clean_text(query)

    query_words = query.split()

    corrected_words = []

    for word in query_words:
        corrected_word = find_nearest_word(word, words_set)

        corrected_words.append(corrected_word)

    corrected_query = " ".join(corrected_words)

    return corrected_query

In [418]:
spell_checking("festivsl funders", words_set)

'festival founders'

In [419]:
spell_checking("Hello World. You're wild!", words_set)

'sell world your will'

In [420]:
spell_checking("aaa bbb cccc dd", words_set)

'saw bob check did'

In [421]:
spell_checking("this should not change", words_set)

'this should not change'

### 3. Wildcard Queries

You will implement a function that handles wildcard queries using Permuterm or K-gram
(K=2) indexing. Your function needs to be able to support queries with one or two * symbols. To this end, you
might need to post-filter false positive outcomes. You cannot use Regex.


In [422]:
class TrieNode:
    """A node in the trie structure"""

    def __init__(self, char):
        # the character stored in this node
        self.char = char

        # whether this can be the end of a word
        self.is_end = False

        # a dictionary of child nodes
        # keys are characters, values are nodes
        self.children = {}

In [423]:
class Trie(object):
    def __init__(self):
        self.root = TrieNode("")

    def insert(self, word):
        """Insert a word into the trie"""
        node = self.root

        # Loop through each character in the word
        # Check if there is no child containing the character, create a new child for the current node
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                # If a character is not found,
                # create a new node in the trie
                new_node = TrieNode(char)
                node.children[char] = new_node
                node = new_node

        # Mark the end of a word
        node.is_end = True

    def dfs(self, node, result, prefix=""):
        """
        Depth-first traversal of the trie
        Find and print all worlds of the tree
        """
        if node.is_end:
            result.append(prefix + node.char)

        for child in node.children.values():
            self.dfs(child, result, prefix + node.char)

        return result

    def query(self, x):
        """Given an input (a prefix), retrieve all words stored in
        the trie with that prefix
        """

        node = self.root

        # Check if the prefix is in the trie
        for char in x:
            # print(node.children)
            if char in node.children:
                node = node.children[char]
            else:
                # cannot found the prefix, return empty list
                return []

        # Traverse the trie to get all candidates
        result = []

        self.dfs(node, result, x[:-1])
        
        return result

In [424]:
test_trie = Trie()
test_trie.insert("he")
test_trie.insert("hell")
test_trie.insert("hello")
test_trie.insert("yellow")

In [425]:
result = []
test_trie.dfs(test_trie.root, result)
result

['he', 'hell', 'hello', 'yellow']

In [426]:
test_trie.query("")

['he', 'hell', 'hello', 'yellow']

In [427]:
test_trie.query("hel")

['hell', 'hello']

In [428]:
def permuterm_vocab(word):
    """Take a world, insert $ and make all possible combinations by circular shifting the word"""

    result = []

    for i in range(len(word) + 1):
        result.append(word[i:] + "$" + word[:i])

    return result

In [429]:
permuterm_vocab("hello")

['hello$', 'ello$h', 'llo$he', 'lo$hel', 'o$hell', '$hello']

We can read words from file and insert them directly to trie
to improve space complexity;
but we already have them in words_set.

In [430]:
def build_permuterm_index_tree(words_set):
    trie = Trie()

    for word in words_set:
        terms = permuterm_vocab(word)
        
        for term in terms:
            trie.insert(term)

    return trie

In [431]:
permuterm_index_tree = build_permuterm_index_tree(words_set)

permuterm_index_tree.query("")

['private$',
 'prison$',
 'prisoners$',
 'prietor$pro',
 'price$',
 'prices$',
 'pril$a',
 'pring$s',
 'prince$',
 'property$',
 'proprietor$',
 'proposal$',
 'provide$',
 'proved$ap',
 'proven$',
 'problem$',
 'problems$',
 'probably$',
 'proceeds$',
 'produce$',
 'pros$',
 'profit$',
 'predictable$un',
 'pretty$',
 'president$',
 'practiced$',
 'practice$',
 'p$scoo',
 'p$sto',
 'p$stee',
 'p$soa',
 'p$la',
 'p$lim',
 'p$hel',
 'p$u',
 'p$pul',
 'p$cu',
 'p$chea',
 'per$',
 'per$chea',
 'per$newspa',
 'per$pa',
 'persons$',
 'personal$',
 'perty$pro',
 'perthe$pa',
 'percent$',
 'perimenting$ex',
 'period$',
 'permitted$',
 'perate$des',
 'perationninety$o',
 'perfect$',
 'pes$sha',
 'pes$ho',
 'penny$',
 'pen$hap',
 'pened$hap',
 'penter$car',
 'ped$stop',
 'ped$slum',
 'ped$drop',
 'ped$jum',
 'ped$flip',
 'people$',
 'people$drinks',
 'pe$ho',
 'pecter$s',
 'pewed$s',
 'pears$ap',
 'ph$autogra',
 'phone$',
 'phone$saxo',
 'phonist$saxo',
 'phoenix$',
 'physema$em',
 'parking$',
 '

In [432]:
permuterm_index_tree.query("y$fr")

['y$freewa', 'y$friendl']

In [433]:
permuterm_index_tree.query("her$")

['her$',
 'her$ot',
 'her$eit',
 'her$hig',
 'her$anot',
 'her$rat',
 'her$toget',
 'her$whet']

In [434]:
permuterm_index_tree.query("$aaa")

[]

In [435]:
def index_of_asterisks(word):
    """find index of all asterisks in an string"""

    result = []
    
    for i in range(len(word)):
        if word[i] == "*":
            result.append(i)
    
    return result

In [436]:
asterisks1 = index_of_asterisks("not even a single asterisk")
asterisks2 = index_of_asterisks("*")
asterisks3 = index_of_asterisks("*aaa*aa*aa*")
print(asterisks1, len(asterisks1))
print(asterisks2, len(asterisks2))
print(asterisks3, len(asterisks3))

[] 0
[0] 1
[0, 4, 7, 10] 4


In [437]:
def remove_dollar_sign(term):
    index = term.find("$")

    return term[index + 1 :] + term[:index]

In [438]:
def single_asterisk_query(term, trie: Trie):
    """Find all words that match the expression witch has one asterisk"""
    asterisks = index_of_asterisks(term)

    if len(asterisks) != 1:
        raise Exception("number of asterisks must be 1")

    term = term + "$"

    query_term = term[asterisks[0] + 1 :] + term[: asterisks[0]]

    return [remove_dollar_sign(term) for term in trie.query(query_term)]

In [439]:
single_asterisk_query("fin*y", permuterm_index_tree)

['finally']

In [440]:
single_asterisk_query("a*a", permuterm_index_tree)

['altadena', 'arcadia', 'arizona', 'area']

In [441]:
single_asterisk_query("*na", permuterm_index_tree)

['altadena', 'arizona', 'pasadena', 'donna']

In [442]:
single_asterisk_query("ab*", permuterm_index_tree)

['about', 'able', 'abuse']

In [443]:
def double_asterisk_query(term, trie):
    """Find all words that match the expression witch has two asterisks"""
    asterisks = index_of_asterisks(term)

    if len(asterisks) != 2:
        raise Exception("number of asterisks must be 2")

    # Remove letters between asterisks and one of stars
    new_term = term[: asterisks[0]] + term[asterisks[1] :]

    phrase_between_asterisks = term[asterisks[0] + 1 : asterisks[1]]

    initial_result = single_asterisk_query(new_term, trie)

    # Check if letters between asterisks exist in the right place
    result = []

    for word in initial_result:
        index = word.find(
            phrase_between_asterisks,
            asterisks[0],
            len(word) - (len(term) - asterisks[1]) + 1,
        )

        if index != -1:
            result.append(word)

    return(result)

In [444]:
double_asterisk_query("f*na*y", permuterm_index_tree)

['finally']

In [445]:
single_asterisk_query("f*y", permuterm_index_tree)

['freeway', 'friendly', 'finally', 'facility']

In [446]:
double_asterisk_query("s*a*y", permuterm_index_tree)

['say',
 'saturday',
 'safely',
 'stray',
 'stay',
 'stationery',
 'steadily',
 'sunday']

In [447]:
double_asterisk_query("a*m*y", permuterm_index_tree)

[]

In [448]:
single_asterisk_query("fin*", permuterm_index_tree)

['find', 'fine', 'fines', 'finally']

In [449]:
def wildcard_query(term, trie):
    asterisks = index_of_asterisks(term)

    if len(asterisks) == 0:
        # Return all words that term is their prefix
        return single_asterisk_query(term + "*", trie)
    
    elif len(asterisks) == 1:
        return single_asterisk_query(term, trie)
    
    elif len(asterisks) == 2:
        return double_asterisk_query(term, trie)
    
    else:
        raise Exception("number of asterisks must be 2 or less")

In [450]:
wildcard_query("n*b*y", permuterm_index_tree)

['nobody', 'nearby']

In [451]:
wildcard_query("fin", permuterm_index_tree)

['find', 'fine', 'fines', 'finally']

In [452]:
wildcard_query("s*e*e", permuterm_index_tree)

['sometime', 'sidewalksnoise', 'silverware']

In [453]:
wildcard_query("sa*e", permuterm_index_tree)

['sale', 'same', 'saxophone', 'save']